In [2]:
import reaction_parallel_cython
import numpy as np

In [37]:
from numba import jit

In [16]:
N = 10000

cellsize = 100

PDTYPE = [('pos', np.float64, (3,)),
          ('vel', np.float64, (3,)),
          ('E', 'f8'), 
          ('id', 'i8')]

CELLDTYPE = [('id', 'i4'), 
            ('index', np.int32, (3,)),
            ('normal', np.float64, (3,)) ]

particle = np.zeros(N, dtype=PDTYPE)
# cell = np.zeros((cellsize, cellsize, cellsize), dtype=CELLDTYPE)

pos = np.random.rand(N, 3)*99
vel = np.random.rand(N, 3)
vel = vel/np.linalg.norm(vel)

# 填充粒子数据
particle['pos'] = pos
particle['vel'] = vel
particle['E'] = 100
particle['id'] = 1



In [8]:
particle[11]

([39.52086473, 82.60724675, 25.57822548], [0.00778588, 0.00106172, 0.00934082], 100., 1)

In [29]:
particle[11]['vel']

array([0.00707488, 0.00276015, 0.00210363])

In [5]:
np.random.rand(N, 3).shape

(10000, 3)

In [17]:
cell = np.zeros((cellsize, cellsize, cellsize), dtype=CELLDTYPE)

nn = np.array([5, 6, 5])
nn = nn/np.linalg.norm(nn)

cell['id'] = 0
cell['index'] = np.array([1,2,3])
cell['normal'] = nn

cell[:,:,50:]['id'] = 1

In [18]:
cell[50, 50, 15]

(0, [1, 2, 3], [0.53916387, 0.64699664, 0.53916387])

In [15]:
cell.shape

(100, 100, 100)

In [20]:
a = reaction_parallel_cython.particle_parallel(particle, cell)

In [24]:
%timeit reaction_parallel_cython.particle_parallel(particle, cell)

449 μs ± 9.68 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [23]:
print(a)

[0.         0.01286841 0.02126602 ... 0.         0.         0.        ]


In [46]:
@jit(nopython=True, parallel=True)
def particle_parallel_numba(particles, cell):

    dot_product = np.zeros(particles.shape[0], dtype=np.double)

    for i in range(particles.shape[0]):
        cellijk = particles[i]['pos'].astype(np.int32)

        if cell[cellijk[0],cellijk[1], cellijk[2]]['id'] == 1:
            dot_product[i] = np.dot(particles[i]['vel'], cell[cellijk[0],cellijk[1], cellijk[2]]['normal'])

    return dot_product

In [38]:

def particle_parallel(particles, cell):

    dot_product = np.zeros(particles.shape[0], dtype=np.double)

    for i in range(particles.shape[0]):
        celli = particles[i]['pos'][0].astype(np.int32)
        cellj = particles[i]['pos'][1].astype(np.int32)
        cellk = particles[i]['pos'][2].astype(np.int32)

        if cell[celli, cellj, cellk]['id'] == 1:
            dot_product[i] = np.dot(particles[i]['vel'], cell[celli, cellj, cellk]['normal'])

    return dot_product

In [47]:
%timeit particle_parallel_numba(particle, cell)

103 ms ± 5.74 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
